<div class="alert alert-block alert-success">
    <h1 align="center">Plant Classification</h1>
    <h3 align="center">A2NDL POLIMI</h3>

</div>

### 1-Mount Google drive and unzip data

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # mount google drive to /content directory
!unzip -qq "/content/drive/MyDrive/public_data.zip" -d "/content"  # unzip dataset to the /content directory

Mounted at /content/drive
unzip:  cannot find or open /content/drive/MyDrive/public_data.zip, /content/drive/MyDrive/public_data.zip.zip or /content/drive/MyDrive/public_data.zip.ZIP.


### 2-Import Libraries

In [ ]:
import os, datetime, warnings, logging, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_squared_error
from keras import callbacks
from keras import layers
from keras import models
from keras.layers import Dropout
import keras,cv2
from keras import optimizers
from sklearn import preprocessing
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import cv2, glob, random, os, time, shutil, datetime,time, keras
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D,MaxPool2D, BatchNormalization, Activation, Input, Add, Dense, ZeroPadding2D,Flatten, AveragePooling2D, Rescaling
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras import layers, callbacks , metrics
from joblib import Parallel, delayed
from sklearn.metrics import accuracy_score, confusion_matrix

### 3-Fix randomness and hide warnings

In [ ]:
seed = 42
#####
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'
#####
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
#####
np.random.seed(seed)
random.seed(seed)

### 4-Set first parameters of tensorflow

In [ ]:
# Import tensorflow
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print("tensorflow_version",tf.__version__)

tensorflow_version 2.14.0


### 5-Load dataset

In [ ]:
dataset = np.load("/content/public_data.npz", allow_pickle=True)

FileNotFoundError: ignored

### 6-EDA On Dataset

In [ ]:
# Watch keys available in the numpy array
keys = list(dataset.keys())
print('keys in our dataset are: ', keys)
print('*'*100)

# look at data shape
x = dataset["data"]
no_images = x.shape[0]
size_images = x.shape[1:3]
print('Data shape: ',x.shape)
print('*'*100)

# Lool aat labels
y = dataset["labels"]
no_labels = y.shape[0]
print("Labels are : ",y)
print('*'*100)

# Look at bing balanced or imbalanced
_, counts = np.unique(y,return_counts=True) # count occurrence of each item
no_healthy_images = counts[0]
no_unhealthy_images = counts[1]

# pass variables to a dictionary to be used as dataframe for a better show
info_table_dict = {"no_images":no_images, "image_width": size_images[0],"image_length": size_images[1], "no_labels":no_labels,
                   "no_healthy_images":no_healthy_images,"percentage%":no_healthy_images*100/no_labels,
                   "no_unhealthy_images":no_unhealthy_images,"percentage %":no_unhealthy_images*100/no_labels }
info_table = pd.DataFrame(info_table_dict, index =['value'])
info_table

In [ ]:
# Show histogram of y(labels) to investigate wheather dataset is balanced or not
plt.figure(figsize=(8,6))
plt.hist(y, bins=3,align='mid')
plt.show()

In [ ]:
# Turn healty and unhealthy labels to two classes of 0 and 1, using labelEncoder
labels_before_encoder = y

# Create a LabelEncoder
label_encoder = preprocessing.LabelEncoder()
# Fit and transform the encoder on the original array
labels_after_encoder = label_encoder.fit_transform(y)

# set y  to new values
y = labels_after_encoder


print(f"before label encoding, list is like this :{labels_before_encoder}. And, unique items are: {np.unique(labels_before_encoder)}")
print(f"after label encoding, list is like this :{labels_after_encoder}. And, unique items are: {np.unique(labels_after_encoder)}")

In [ ]:
# Since we want to use softamx activation in last layer of each model
# we must turn y to categorical labels
y = tfk.utils.to_categorical(y,2)
print(f"after label encoding, list is like this :{labels_after_encoder}. And, unique items are: {np.unique(labels_after_encoder)}")
print(f"after turning to categorical, list is like this :{y}. And, unique items are: {np.unique(y)}")

### 7_Show images

In [ ]:
import matplotlib.pyplot as plt

# A function for displaying images in batches
def show_images(dataset, labels, batch_no, no_images_per_batch, num_cols=10):
    '''input: dataset wiht shape(None,width,lenght,channel),
              labels with this format [[1 0]
                                       [1 0]],
              batch_no says which section/batch of your data you want to show,
              no_images_per_batch says how many images you want to put into one batch
       output: show images you want'''

    start_index = batch_no * no_images_per_batch

    end_index = min((batch_no + 1) * no_images_per_batch, len(dataset))

    num_rows = (end_index - start_index + num_cols - 1) // num_cols
    fig, axes = plt.subplots(num_rows, num_cols)
    label = tf.argmax(labels, axis=-1).numpy()

    for i, ax in enumerate(axes.ravel()):
        if start_index + i < end_index:
            image = dataset[start_index + i]
            image = image / 255.0  # Normalize pixel values to [0, 1]
            ax.imshow(image)
            ax.set_title(f"{label[start_index + i]}",fontsize=10, y=-0.5)
            ax.axis('off')

    for i in range(end_index - start_index, num_rows * num_cols):
        fig.delaxes(axes.flatten()[i])

    plt.show()


In [ ]:
show_images(dataset=x,labels =y, batch_no=0,no_images_per_batch=40)

### 8-Splitting data

In [ ]:
# Firstly, we split dataset to training_data and test part. We keep x_test, y_test till the end for evaluating the model
x_training_data, x_test, y_training_data, y_test = train_test_split(
    x,
    y,
    test_size = 0.1,
    shuffle =True,
    random_state=seed,
    stratify=y)# I used stratified since our data is imbalanced


In [ ]:
# Secondly, We split training data into train and validation parts. Training part
# for training and validation part to validate model.
x_train, x_val, y_train, y_val = train_test_split(
    x_training_data,
    y_training_data,
    test_size = len(x_test), # Ensure validation set size matches test set size--> Why??
    random_state=seed,
    shuffle =True,
    stratify=y_training_data)

In [ ]:
# Print the shapes of different datsets after splitting the main dataset into training, validation, testing parts
print('X_train.shape:',x_train.shape,'y_train.shape:', y_train.shape)
print('X_val.shape:',x_val.shape, 'y_val.shape:',y_val.shape)
print('X_test.shape:',x_test.shape,'y_test.shape', y_test.shape)

### 9-Watch data distribution

In [ ]:
# Create three separate subplots to visualize the sorted target variable values for the training, validation, and test sets

# Create a figure for the training labels subplot
plt.figure(figsize=(20, 2))
plt.scatter((np.arange(y.shape[0])), np.sort(tf.argmax(y, axis=-1), axis=0))
plt.title('Training labels')
plt.xticks([])  # Remove x-axis ticks
plt.grid(0.2)   # Add a grid with opacity 0.2 for reference
plt.ylim([-0.1, 1.1])  # Set the y-axis limits to [0.1, 1.1]

# Create a figure for the validation labels subplot
plt.figure(figsize=(20, 2))
plt.scatter(np.arange(y_val.shape[0]), np.sort(tf.argmax(y_val, axis=-1), axis=0))
plt.title('Validation labels')
plt.xticks([])  # Remove x-axis ticks
plt.grid(0.2)   # Add a grid with opacity 0.2 for reference
plt.ylim([-0.1, 1.1])  # Set the y-axis limits to [0.1, 1.1]

# Create a figure for the test labels subplot
plt.figure(figsize=(20, 2))
plt.scatter(np.arange(y_test.shape[0]), np.sort(tf.argmax(y_test, axis=-1), axis=0))
plt.title('Test labels')
plt.xticks([])  # Remove x-axis ticks
plt.grid(0.2)   # Add a grid with opacity 0.2 for reference
plt.ylim([-0.1, 1.1])  # Set the y-axis limits to [0.1, 1.1]

# Display the plots
plt.show()

### 10- Feature Scaling

In [ ]:
x_train = x_train/255.0
x_val = x_val/255.0
x_test = x_test/255.0